In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
import os
import time
import cv2

In [4]:
from ultralytics import YOLO

In [7]:
model = YOLO('./runs/detect/train/weights/best.pt')  # load a custom model

In [8]:
VIDEOS_DIR = os.path.join('.', 'videos')

video_path = "D:/AI-Anchor/videodataset/NTV.mp4"
video_path_out = '{}_out.mp4'.format(video_path)

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
H, W, _ = frame.shape
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))


threshold = 0.8

while ret:

    results = model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    out.write(frame)
    ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 1 anchor, 45.1ms
Speed: 4.0ms preprocess, 45.1ms inference, 494.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 43.1ms
Speed: 4.0ms preprocess, 43.1ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 42.6ms
Speed: 4.0ms preprocess, 42.6ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 43.1ms
Speed: 4.0ms preprocess, 43.1ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 42.6ms
Speed: 3.0ms preprocess, 42.6ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 42.1ms
Speed: 2.5ms preprocess, 42.1ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 22.1ms
Speed: 3.5ms preprocess, 22.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 anchor, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 3.0ms postprocess per image at shape (1, 3, 

In [ ]:
import os
import cv2

# Define the directory for saving videos
VIDEOS_DIR = os.path.join('.', 'videos')

# Path of the input video
video_path = "D:/AI-Anchor/videodataset/Eng.mp4"

# Output video path
video_path_out = '{}_out.mp4'.format(video_path)

# Open the input video
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
H, W, _ = frame.shape

# Initialize the output video writer
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

# Threshold for detection confidence
threshold = 0.8

# Define the class you want to crop (example: 'person')
target_class = 'person'

while ret:
    # Perform object detection on the frame
    results = model(frame)[0]

    # List to store the cropped frames
    cropped_frames = []

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold and results.names[int(class_id)] == target_class:
            # Crop the frame based on the bounding box coordinates
            cropped_frame = frame[int(y1):int(y2), int(x1):int(x2)]
            cropped_frames.append(cropped_frame)

            # Draw bounding box and class label on the original frame
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    # Write the original frame with bounding boxes to the output video
    out.write(frame)

    # Write the cropped frames to individual files
    for i, cropped_frame in enumerate(cropped_frames):
        cv2.imwrite(os.path.join(VIDEOS_DIR, f'{target_class}_{i}.jpg'), cropped_frame)

    # Read the next frame
    ret, frame = cap.read()

# Release video capture and writer
cap.release()
out.release()
cv2.destroyAllWindows()
